In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [34]:
df = pd.read_csv('../DATA/Advertising.csv')
df.head()

,TV,radio,newspaper,sales
0,230.1,37.8,69.2,22.1
1,44.5,39.3,45.1,10.4
2,17.2,45.9,69.3,9.3
3,151.5,41.3,58.5,18.5
4,180.8,10.8,58.4,12.9


----
----
----
## Разбиваем данные на обучающий и тестовый наборы данных - Train | Test Split

0. Очищаем и масштабируем данные X и y (при необходимости)
1. Разбиваем данные на обучающий и тестовый наборы данных - как для X, так и для y
2. Обучаем объект Scaler на обучающих данных X
3. Применяем масштабирование (scale) для тестовых данных X
4. Создаём модель
5. Обучаем модель на обучающих данных X
6. Оцениваем модель на тестовых данных X (создавая предсказания и сравнивая их с Y_test)
7. Уточняем параметры модели, повторяя шаги 5 и 6

---
#### 0. Обозначаем Х и Y.

In [35]:
X = df.drop('sales', axis=1)
y = df['sales']

#### 1. Разбиваем данные на train & test.

In [36]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=101)

#### 2. Маштабирование данных (если нужно).

In [37]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

#### 3. Создаём модель.

In [38]:
from sklearn.linear_model import Ridge

model = Ridge(alpha=100)
model.fit(X_train_scaled, y_train)

y_pred = model.predict(X_test_scaled)

In [39]:
from sklearn.metrics import mean_squared_error

MSE = mean_squared_error(y_test,y_pred)
RMSE = np.sqrt(MSE)

print(f'MSE: {MSE}')
print(f'RMSE: {RMSE}\n')

MSE: 7.341775789034124
RMSE: 2.709571144855607



#### 3.1. Вторая модель.

In [40]:
model_two = Ridge(alpha=1)
model_two.fit(X_train_scaled, y_train)

y_pred = model_two.predict(X_test_scaled)

MSE = mean_squared_error(y_test,y_pred)
RMSE = np.sqrt(MSE)

print(f'MSE: {MSE}')
print(f'RMSE: {RMSE}\n')

MSE: 2.3190215794287514
RMSE: 1.5228334050147283



----
----
----
## Обучение - валидация - тестирование (Train | Validation | Test Split Procedure)

Этот подход ещё называют методом с задержкой ("hold-out"), потому что здесь мы не настраиваем параметры модели на финальном тестовом наборе данных, а используем тестовые данные *только* для оценки финального результата.

0. Очищаем и масштабируем данные X и y (при необходимости)
1. Разбиваем данные на обучающий, оценочный и тестовый наборы данных - как для X, так и для y
2. Обучаем объект Scaler на обучающих данных X
3. Масштабируем (scale) оценочные данные X
4. Создаём модель
5. Обучаем модель на обучающих данных X
6. Оцениваем модель на оценочных данных X (создавая предсказания и сравнивая их с Y_eval)
7. Уточняем параметры модели, повторяя шаги 5 и 6
8. Вычисляем финальные метрики на тестовом наборе данных (после этого уже нельзя возвращаться и делать уточнения!)

In [41]:
X = df.drop('sales', axis=1)
y = df['sales']

In [42]:
from sklearn.model_selection import train_test_split

X_train, X_other, y_train, y_other = train_test_split(X, y, test_size=0.3, random_state=101)

In [43]:
#test_size = 0.5 (50% из 30% other ---> test_size = 15% от исходного объёма данных)

X_eval, X_test, y_eval, y_test = train_test_split(X_other,y_other, test_size=0.5, random_state=101)

print(X_train.shape)
print(X_eval.shape)
print(X_test.shape)

(140, 3)
(30, 3)
(30, 3)


In [44]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(X_train)

X_train = scaler.transform(X_train)
X_eval = scaler.transform(X_eval)
X_test = scaler.transform(X_test)

In [45]:
from sklearn.linear_model import Ridge

model_one = Ridge(alpha=100)
model.fit(X_train, y_train)

y_eval_pred = model.predict(X_eval)

MSE = mean_squared_error(y_eval,y_eval_pred)
RMSE = np.sqrt(MSE)

print(f'MSE: {MSE}')
print(f'RMSE: {RMSE}\n')

MSE: 7.320101458823866
RMSE: 2.7055686017589475



In [47]:
model_two = Ridge(alpha=1)
model_two.fit(X_train, y_train)

y_eval_pred_two = model_two.predict(X_eval)

MSE = mean_squared_error(y_eval,y_eval_pred_two)
RMSE = np.sqrt(MSE)

print(f'MSE: {MSE}')
print(f'RMSE: {RMSE}\n')

MSE: 2.3837830750569866
RMSE: 1.5439504768796786



#### Финальная оценка модели на X_test.

In [48]:
y_test_pred = model_two.predict(X_test)

MSE = mean_squared_error(y_test,y_test_pred)
RMSE = np.sqrt(MSE)

print(f'MSE: {MSE}')
print(f'RMSE: {RMSE}\n')

MSE: 2.254260083800517
RMSE: 1.5014193564093001



----
----
----
## Кросс-валидация с помощью cross_val_score

----

<img src="grid_search_cross_validation.png">

----

In [49]:
df = pd.read_csv('../DATA/Advertising.csv')
df.head()

,TV,radio,newspaper,sales
0,230.1,37.8,69.2,22.1
1,44.5,39.3,45.1,10.4
2,17.2,45.9,69.3,9.3
3,151.5,41.3,58.5,18.5
4,180.8,10.8,58.4,12.9


In [50]:
X = df.drop('sales', axis=1)
y = df['sales']

In [52]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=101)

In [53]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(X_train)

X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [54]:
from sklearn.linear_model import Ridge

model = Ridge(alpha=100)

In [55]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(estimator=model, X=X_train, y=y_train, scoring='neg_mean_squared_error', cv=5)

In [56]:
# значения указанной метрики 'neg_mean_squared_error' для каждой их 5 итерации.
scores

array([ -9.32552967,  -4.9449624 , -11.39665242,  -7.0242106 ,
        -8.38562723])

In [57]:
abs(np.mean(scores))

np.float64(8.215396464543606)

In [64]:
# 2 модель

model = Ridge(alpha=1)
scores = cross_val_score(estimator=model, X=X_train, y=y_train, scoring='neg_mean_squared_error', cv=5)
print(scores)
print(abs(np.mean(scores)))

[-3.15513238 -1.58086982 -5.40455562 -2.21654481 -4.36709384]
3.3448392965306946


In [66]:
model.fit(X_train,y_train)

y_test_pred = model.predict(X_test)

MSE = mean_squared_error(y_test,y_test_pred)
RMSE = np.sqrt(MSE)

print(f'MSE: {MSE}')
print(f'RMSE: {RMSE}\n')

MSE: 2.3190215794287514
RMSE: 1.5228334050147283



----
----
----

# Кросс-валидация с помощью cross_validate

Функция cross_validate отличается от cross_val_score двумя аспектами:

эта функция позволяет использовать для оценки несколько метрик;

она возвращает не только оценку на тестовом наборе (test score), но и словарь с замерами времени обучения и скоринга, а также - опционально - оценки на обучающем наборе и объекты estimator.

В случае одной метрики для оценки, когда параметр scoring является строкой string, вызываемым объектом callable или значением None, ключи словаря будут следующими:
        
        - ['test_score', 'fit_time', 'score_time']

А в случае нескольких метрик для оценки, возвращаемый словарь будет содержать следующие ключи:

    ['test_<scorer1_name>', 'test_<scorer2_name>', 'test_<scorer...>', 'fit_time', 'score_time']

return_train_score по умолчанию принимает значение False, чтобы сэкономить вычислительные ресурсы. Чтобы посчитать оценки на обучающем наборе, достаточно установить этот параметр в значение True.

In [91]:
df = pd.read_csv('../DATA/Advertising.csv')

## Создаём X и y
X = df.drop('sales',axis=1)
y = df['sales']

# Делаем разбиение на TRAIN и TEST
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=101)

# Масштабируем данные (SCALE)
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [92]:
from sklearn.model_selection import cross_validate
from sklearn.linear_model import Ridge

model = Ridge(alpha=100)
scores = cross_validate(model, X_train, y_train, 
                         scoring=['neg_mean_squared_error','neg_mean_absolute_error',], cv=10)

scores = pd.DataFrame(scores)
print(f'Mean:\n{scores.mean()}')
scores

Mean:
fit_time                        0.001183
score_time                      0.005055
test_neg_mean_squared_error    -7.565121
test_neg_mean_absolute_error   -2.083773
dtype: float64


,fit_time,score_time,test_neg_mean_squared_error,test_neg_mean_absolute_error
0,0.001415,0.002064,-6.060671,-1.810212
1,0.001407,0.001034,-10.627031,-2.541958
2,0.000930,0.041393,-3.993426,-1.469594
3,0.000826,0.000681,-5.009494,-1.862769
4,0.000595,0.000545,-9.141800,-2.520697
5,0.000513,0.001307,-13.086256,-2.459995
6,0.004477,0.001952,-3.839405,-1.451971
7,0.000730,0.000594,-9.058786,-2.377395
8,0.000495,0.000506,-9.055457,-2.443344
9,0.000444,0.000473,-5.778882,-1.899797


In [93]:
model = Ridge(alpha=1)
scores = cross_validate(model, X_train, y_train, 
                         scoring=['neg_mean_squared_error','neg_mean_absolute_error',], cv=10)

scores = pd.DataFrame(scores)
print(f'Mean:\n{scores.mean()}')
scores

Mean:
fit_time                        0.001069
score_time                      0.000965
test_neg_mean_squared_error    -3.323018
test_neg_mean_absolute_error   -1.308467
dtype: float64


,fit_time,score_time,test_neg_mean_squared_error,test_neg_mean_absolute_error
0,0.001651,0.001159,-2.962508,-1.457174
1,0.001625,0.001551,-3.057378,-1.555308
2,0.000891,0.001001,-2.173740,-1.238770
3,0.000895,0.001257,-0.833034,-0.768938
4,0.001719,0.000741,-3.464018,-1.434489
5,0.001015,0.000721,-8.232647,-1.494316
6,0.000808,0.000833,-1.905864,-1.081362
7,0.000762,0.000784,-2.765048,-1.250011
8,0.000713,0.000724,-4.989505,-1.580971
9,0.000609,0.000877,-2.846438,-1.223326


In [94]:
model.fit(X_train, y_train)

y_test_pred = model.predict(X_test)

MSE = mean_squared_error(y_test,y_test_pred)
RMSE = np.sqrt(MSE)

print(f'MSE: {MSE}')
print(f'RMSE: {RMSE}\n')

MSE: 2.3190215794287514
RMSE: 1.5228334050147283

